In [3]:
# List of TTPs to check
ttp_list = [
    'T1194', 'T1088', 'T1346', 'T1065', 'T1500', 'T1311', 'T1432', 'T1002', 'T1043', 
    'T1433', 'T1329', 'T1312', 'T1179', 'T1517', 'T1093', 'T0872', 'T1079', 'T1009', 
    'T0840', 'T1099', 'T1053.004', 'T1045', 'T1426', 'T1100', 'T1512', 'T1022', 'T1438', 
    'T1420', 'T1532', 'T1508', 'T0871', 'T1533', 'T1476', 'T1077', 'T1191', 'T1402', 
    'T1341', 'T1024', 'T1117', 'T1047.001', 'T1345', 'T1004', 'T1497.004', 'T1064', 
    'T1247', 'T1089', 'T1249', 'T1086', 'T1260', 'T1061', 'T0853', 'T1143', 'T0847', 
    'T1122', 'T1084', 'T1351', 'T1268', 'T0807', 'T1023', 'T1107', 'T1444', 'T1660', 
    'T0831', 'T0809', 'T1081', 'T1101', 'T1437', 'T1076', 'T1503.004', 'T1035', 'T1412', 
    'T1308', 'T1328', 'T1013', 'T1031', 'T1060', 'T1193', 'T1085', 'T0855', 'T1513', 
    'T1116', 'T1168', 'T1094', 'T1447', 'T1138', 'T1063', 'T1183', 'T1032', 'T1158', 
    'T1418', 'T1502', 'T1337', 'T1145', 'T1575', 'T1347', 'T1503.003', 'T1503', 'T1429', 
    'T1544', 'T1050', 'T1108', 'T1073', 'T1015', 'T1038', 'T1192', 'T1362', 'T1066'
]

In [4]:
# List of STIX file versions with raw URLs for each version
versions = [
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-1.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-2.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-3.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-4.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-5.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-6.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-7.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-8.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-9.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-10.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-11.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-12.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-13.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-14.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-15.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-16.0.json"
]


In [5]:
import json
import requests

def get_attack_stix_data(version_url):
    """
    Fetches STIX data for a given MITRE ATT&CK version.
    
    Args:
        version_url (str): URL for the MITRE ATT&CK STIX data.
    
    Returns:
        dict: Parsed JSON data from the URL.
    """
    response = requests.get(version_url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Error fetching data from {version_url}. Status code: {response.status_code}")

def find_ttp_status_in_version(stix_data, ttp_id):
    """
    Checks if a specific TTP ID exists in the STIX data and whether it is deprecated.

    Args:
        stix_data (dict): Parsed STIX JSON data.
        ttp_id (str): The TTP ID to search for.

    Returns:
        tuple: (bool, bool, str) where:
            - bool: True if the TTP is present.
            - bool: True if the TTP is deprecated.
            - str: The TTP's name (if present).
    """
    for obj in stix_data.get('objects', []):
        if obj.get('type') == 'attack-pattern':
            # Check for TTP ID in external references
            external_references = obj.get('external_references', [])
            for reference in external_references:
                if ttp_id == reference.get('external_id'):
                    # Check if deprecated
                    deprecated = obj.get('x_mitre_deprecated', False)
                    ttp_name = obj.get('name', 'Unknown')
                    return True, deprecated, ttp_name
    return False, False, None

def track_ttp_across_versions(versions, ttp_id):
    """
    Tracks the presence of a TTP across multiple ATT&CK versions and when it becomes deprecated.

    Args:
        versions (list): List of URLs of different ATT&CK STIX versions.
        ttp_id (str): The TTP ID to track.

    Returns:
        dict: A dictionary containing the first appearance version, deprecated version, and TTP name.
    """
    first_appearance = None
    deprecated_version = None
    ttp_name = None
    
    for version_url in versions:
        stix_data = get_attack_stix_data(version_url)
        present, deprecated, name = find_ttp_status_in_version(stix_data, ttp_id)
        
        # Update when TTP is first found
        if present and first_appearance is None:
            first_appearance = version_url
            ttp_name = name
        
        # Update when TTP is marked deprecated
        if present and deprecated:
            deprecated_version = version_url
    
    return {
        'ttp_id': ttp_id,
        'ttp_name': ttp_name,
        'first_appearance': first_appearance,
        'deprecated_version': deprecated_version
    }

# List of STIX file versions with raw URLs for each version
versions = [
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-1.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-2.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-3.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-4.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-5.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-6.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-7.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-8.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-9.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-10.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-11.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-12.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-13.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-14.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-15.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-16.0.json"
]

# Loop over the list of TTPs
for ttp_id in ttp_list:
    # Track the TTP across versions
    result = track_ttp_across_versions(versions, ttp_id)

    # Output the result for each TTP
    print(f"TTP ID: {result['ttp_id']}")
    print(f"TTP Name: {result['ttp_name']}")
    print(f"First Appearance in: {result['first_appearance']}")
    print(f"Deprecated in: {result['deprecated_version']}")
    print("-" * 50)

TTP ID: T1194
TTP Name: Spearphishing via Service
First Appearance in: https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-2.0.json
Deprecated in: None
--------------------------------------------------
TTP ID: T1088
TTP Name: Bypass User Account Control
First Appearance in: https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-1.0.json
Deprecated in: None
--------------------------------------------------
TTP ID: T1346
TTP Name: None
First Appearance in: None
Deprecated in: None
--------------------------------------------------
TTP ID: T1065
TTP Name: Uncommonly Used Port
First Appearance in: https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-1.0.json
Deprecated in: None
--------------------------------------------------
TTP ID: T1500
TTP Name: Compile After Delivery
First Appearance in: https://raw.githubusercontent.com/m